<a href="https://colab.research.google.com/github/mcd-unison/material-programacion/blob/main/intro-pandas-num.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="100">
</p>


# Curso Propedéutico en *Programación*

# Uso de pandas en datos numéricos y temporales


**Julio Waissman Vilanova**
</center>

In [ ]:
import pandas as pd

## Leyendo y revisando los datos

Para ejemplificar el uso de columnas numéricas, horas y fechas vamos a utilizar un conjunto de datos que puso generosamente a nuestra disposición [Hector Alberto Gutierrez Ibarra](hector.gutierrez@cenace.gob.mx) de la gerencia noroeste del *Centro Nacional de Control de Energía (CENACE)*.

Vamos primero pegandole un ojo a los datos:

In [ ]:
path_datos = "https://github.com/juliowaissman/curso-python-cd/raw/main/datos/caso_zc_hmo.csv.zip"
df = pd.read_csv(path_datos)
df

In [ ]:
df.info()

Lo primero que tenemos que hacer es convertir la variable `Date`en un formato de fechas. En este caso es muy fácil porque la tabla está muy bien formateada, pero suele no ser tan sencillo.

In [ ]:
df['Date'] = pd.to_datetime(df.Date, format="%d/%m/%Y %H:%M")

df

In [ ]:
df.info()

In [ ]:
df.describe()

¿Cuantos días tenemos en esta base? ¿Cual es el primer día y el último?

In [ ]:
print(f"Inicia el {df.Date.min()} y termina el {df.Date.max()}")
print(f"Con una duración de {df.Date.max() - df.Date.min()}")

Hay que tener cuidado porque hay dos tipos fundamentales de formatos de tiempo, y suelen no mesclarse bien en algunas operaciones.

In [ ]:
type(df.Date.min()), type(df.Date.max() - df.Date.min())

Para facilitar el uso de pandas, vamos a pasar la fecha como el indice del dataframe

In [ ]:
# df.index = df.Date
# df.drop(columns='Date', inplace=True)
df.set_index('Date', inplace=True)
df

In [ ]:
df['15/07/2020']

## Gráficas rápidas y furiosas desde pandas

Pandas trae incluidas facilidades para la graficación con el fin de hacer análisis rápidos de nuestras variables, así que vamos aprovechando y haciendo una inspección visual

In [ ]:
ax = df.Demand.plot()

In [ ]:
ax = df.plot(figsize=(12, 20), subplots=True)

In [ ]:
ax = df.Temperature.plot.box()

In [ ]:
ax = df.plot.scatter(
    x='Humidity',
    y='Temperature',
    c='Demand',
    s =40,
    figsize=(12,8)
)

# Generando nuevas variables a partir de las variables conocidas

Generar nuevas variables es relativamente simple, pero hay algunas cosas que son diferentes con numpy y por las cuales hay que tener cuidado.

In [ ]:
df['farenheit'] = (df.Temperature * 9/5) + 32

In [ ]:
df['refri'] = 0
df['refri'] = df.refri.where(df.Temperature < 30, 1)
df.describe()

In [ ]:
df['DiaSemana'] = df.index.weekday
df

In [ ]:
df.rename(
    columns={
        'Demand': 'Demanda',
        'Temperature': 'Temperatura',
        'PrecipIntensity': 'Precipitación',
        'Humidity': 'Humedad',
        'WinSpeed': 'VelocidadViento',
    },
    inplace=True
)
df

## Analizando con regrupamientos

En particular, es interesante poder hacer algun análisis exploratorio utilizando las facilidades que da el uso del manejo de la información temporal que ofrece pandas. En espacial cuando se combina con `groupby` y diferentes funciones de agregación.

Vamos a empezar por ver la demanda promedio por día de la semana:

In [ ]:
ax = df.groupby(df.index.month)[['Demanda']].boxplot(subplots=False, rot=90)

In [ ]:
df.boxplot(column=['Demanda'], by=df.index.month)

In [ ]:
df_semana = df.groupby('DiaSemana').agg(
    {
        'Demanda': ['min', 'max', 'mean', 'median', 'std'],
        'Precipitación': ['min', 'max']
    }
)
df_semana

Ahora vamos a ver que pasa en forma mensual

In [ ]:
ax = df[['Demanda']].groupby(df.index.month).boxplot(subplots=False, rot=90, figsize=(12, 7))

In [ ]:
ax = df[['Temperatura']].groupby(df.index.month).boxplot(subplots=False, rot=90, figsize=(12, 7))

O inclusive por hora del día

In [ ]:
ax = df[['Demanda']].groupby(df.index.hour).boxplot(subplots=False, rot=90, figsize=(12, 7))

In [ ]:
df_hora = df[['Demanda', 'Temperatura']].groupby([df.index.hour, df.index.month_name()]).mean().unstack()
df_hora

In [ ]:
ax = df_hora.Demanda.plot(
    style='o', 
    figsize=(15,7),
    title='Demanda por hora y por mes',
    xlabel='Hora',
    ylabel='Demanda'
)

In [ ]:
ax = df_hora.Temperatura.plot(style='o', figsize=(15,7))